# Chapter 10. Proccesing text
## Notebook for R

### 10.1 Reading and cleaning text

In [129]:
library(glue)
library(magrittr)
my_wiki_text = "<p><b>Communication</b> (from Latin <i>communicare</i>, meaning to share)"
glue(my_wiki_text)

my_wiki_text %<>% gsub("<p>", " ", .) %>% gsub("<b>", " ", .) %>% gsub("</b>", " ", .) %>% gsub("<i>", " ", .) %>% gsub("</i>", " ", .) 
my_wiki_text_2 = gsub('[[:punct:]]','',my_wiki_text) #Remove punctuation
my_wiki_text_2 = tolower(my_wiki_text_2) #Convert to lower case
my_wiki_text_2 = trimws(gsub("\\s+", " ", my_wiki_text_2)) #Remove double spaces 
glue(my_wiki_text_2)

<p><b>Communication</b> (from Latin <i>communicare</i>, meaning to share)

communication from latin communicare meaning to share

In [119]:
library(stringi)
my_wiki_text_3 = stri_replace_all(my_wiki_text, "", regex = "<.+?>")
my_wiki_text_3 = stri_trim(my_wiki_text_3)
glue(my_wiki_text_3)

Communication  (from Latin  communicare , meaning to share)

In [21]:
my_wiki_text_4 = gsub(pattern = "<.*?>", replacement = "", x = my_wiki_text)
glue(my_wiki_text_4)

Communication  (from Latin  communicare , meaning to share)

In [126]:
tweets = "<@born_in_america: My second favorite color is green, I must acknowlege my friends for that!>, <@born_in_britain: My second favourite colour is red, I must aknowledge my friends for that!>"
tweets_unified = gsub(pattern = "acknowlege|aknowledge" , replacement = "acknowledge", x = tweets)
tweets_unified = gsub(pattern = "col.+?r", replacement = "color", x = tweets_unified)
tweets_unified = gsub(pattern = "fav.+?rite" , replacement = "favorite", x = tweets_unified)
glue(tweets)
glue(tweets_unified)

<@born_in_america: My second favorite color is green, I must acknowlege my friends for that!>, <@born_in_britain: My second favourite colour is red, I must aknowledge my friends for that!>

<@born_in_america: My second favorite color is green, I must acknowledge my friends for that!>, <@born_in_britain: My second favorite color is red, I must acknowledge my friends for that!>

In [128]:
users = unlist(strsplit(tweets, " "))
users = gsub("[^[:alnum:]@_]", "", users[grep("(^|[^@\\w])@(\\w{1,15})\\b", users)])
print(users)

[1] "@born_in_america" "@born_in_britain"


In [118]:
users = unlist(strsplit(tweets, " "))
users

[1] "<@born_in_america:" "My"                 "second"            
 [4] "favorite"           "color"              "is"                
 [7] "green,"             "I"                  "must"              
[10] "acknowlege"         "my"                 "friends"           
[13] "for"                "that!>,"            "<@born_in_britain:"
[16] "My"                 "second"             "favourite"         
[19] "colour"             "is"                 "red,"              
[22] "I"                  "must"               "aknowledge"        
[25] "my"                 "friends"            "for"               
[28] "that!>"